In [152]:
from pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn

In [153]:
# read the data
train_data = pd.read_csv("../Datasets/ais_train.csv", delimiter="|")
print(train_data.head())

                  time    cog   sog  rot  heading  navstat       etaRaw  \
0  2024-01-01 00:00:25  284.0   0.7    0       88        0  01-09 23:00   
1  2024-01-01 00:00:36  109.6   0.0   -6      347        1  12-29 20:00   
2  2024-01-01 00:01:45  111.0  11.0    0      112        0  01-02 09:00   
3  2024-01-01 00:03:11   96.4   0.0    0      142        1  12-31 20:00   
4  2024-01-01 00:03:51  214.0  19.7    0      215        0  01-25 12:00   

   latitude  longitude                  vesselId                    portId  
0 -34.74370  -57.85130  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f  
1   8.89440  -79.47939  61e9f3d4b937134a3c4bff1f  634c4de270937fc01c3a7689  
2  39.19065  -76.47567  61e9f436b937134a3c4c0131  61d3847bb7b7526e1adf3d19  
3 -34.41189  151.02067  61e9f3b4b937134a3c4bfe77  61d36f770a1807568ff9a126  
4  35.88379   -5.91636  61e9f41bb937134a3c4c0087  634c4de270937fc01c3a74f3  


In [154]:
# Import necessary modules
import importlib
import preprocess_filter
import feature_engineering_filter

importlib.reload(preprocess_filter)
importlib.reload(feature_engineering_filter)

from preprocess_filter import (
    Sort_by_id,
    Timestamp_to_timedif,
    Create_sequence_vectors,
    Drop_diff_values,
    Normalizer,
)
from feature_engineering_filter import Sequence_training_data

feature_dropper = Drop_diff_values()
normalizer = Normalizer()
id_sorter = Sort_by_id()
timestamp_to_timedif = Timestamp_to_timedif()
create_sequence_vectors = Create_sequence_vectors()
target_creator = Sequence_training_data()

# pipeline = Pipeline(filters=[timestamp_to_timedif,create_sequence_vectors,id_sorter])

# data_processed=pipeline.apply_filters(data=train_data)
# data_processed.to_csv('processed_data.csv', index=True)

In [155]:
data_cleaned = feature_dropper(train_data)
print(data_cleaned.columns)

Index(['time', 'cog', 'sog', 'rot', 'heading', 'navstat', 'latitude',
       'longitude', 'vesselId'],
      dtype='object')


In [156]:
data_timestamped = timestamp_to_timedif(data_cleaned)
data_timestamped.to_csv("intermediate/data_timestamped.csv", index=True)
# data_timestamped=pd.read_csv("intermediate/data_timestamped.csv",index_col=0)
# print(data_timestamped.head())

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
data_normalized, time_scaler = normalizer(data_timestamped)
print(data_normalized.head())

        cog       sog       rot   heading   navstat  latitude  longitude  \
0  0.788889  0.006843  0.498039  0.172211  0.000000  0.108300   0.316705   
1  0.304444  0.000000  0.474510  0.679061  0.066667  0.477832   0.254259   
2  0.308333  0.107527  0.498039  0.219178  0.000000  0.734385   0.262931   
3  0.267778  0.000000  0.498039  0.277886  0.066667  0.111110   0.919778   
4  0.594444  0.192571  0.498039  0.420744  0.000000  0.706382   0.466656   

                   vesselId                time  time_diff  
0  61e9f3a8b937134a3c4bfdf7 2024-01-01 00:00:25    22123.0  
1  61e9f3d4b937134a3c4bff1f 2024-01-01 00:00:36     1980.0  
2  61e9f436b937134a3c4c0131 2024-01-01 00:01:45     1372.0  
3  61e9f3b4b937134a3c4bfe77 2024-01-01 00:03:11     3603.0  
4  61e9f41bb937134a3c4c0087 2024-01-01 00:03:51    85735.0  


In [160]:
data_sequenced = create_sequence_vectors(data_normalized)
data_sequenced.to_csv("intermediate/data_sequenced.csv", index=True)
# data_sequenced=pd.read_csv("intermediate/data_sequenced.csv",index_col=0)
# print(data_sequenced.head())

In [161]:
# create sequence training data
sequence_list, target_list = target_creator(data_sequenced, time_scaler)
# sequence_training_data.to_csv('intermediate/sequence_training_data.csv', index=True)
# sequence_training_data=pd.read_csv("intermediate/sequence_training_data.csv",index_col=0)
# print(sequence_training_data.head())
print(sequence_list.shape)
np.save("intermediate/sequence_list.npy", sequence_list)
np.save("intermediate/target_list.npy", target_list)

/Users/kristofferseyffarth/Downloads/lokalFiles/emner/Maskinlæring/Gruppe/ML-Group-Project/src/feature_engineering_filter.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sequences = row[0]


TypeError: 'float' object is not subscriptable

In [162]:
print(sequence_list.shape)

NameError: name 'sequence_list' is not defined

In [ ]:
from sklearn.model_selection import train_test_split

# Create train and test sets
X = sequence_training_data.drop(["vesselId", "Target"], axis=1)
Y = sequence_training_data["Target"]  # create LST model

train_data, test_data, train_target, test_target = train_test_split(X, Y, test_size=0.9)

In [ ]:
# Train and evaluate model


model = tf.keras.models.Sequential(
    [
        tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(5, 7)),
        tf.keras.layers.LSTM(64, input_shape=(5, 7)),
        tf.keras.layers.Dense(32),
        tf.keras.layers.Dense(2),
    ]
)


model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(sequence_list, target_list, epochs=50, batch_size=64)
model.save("lstm_model.h5")
# loss,accuracy=model.evaluate(test_data,test_target, metrics=["mean_squared_error"])
# print("test accuracy",accuracy,"with loss",loss )

In [ ]:
loss = model.evaluate(sequence_list, target_list)
print("test accuracy", accuracy, "with loss", loss)

In [148]:
# Create test prediction data
import ast

test_data = pd.read_csv("../Datasets/ais_test.csv", delimiter=",")
test_data_cleaned = test_data.drop("scaling_factor", axis=1)

# get sequence_data wherre vesselId is in the test_data
modified_sequences = []

print(data_sequenced.columns)
# Loop over each vesselId in test_data_cleaned
for row in test_data_cleaned.itertuples(10):

    vessel_sequence = data_sequenced.loc[data_sequenced.index == row.vesselId]
    
    print(vessel_sequence["TD5"])

    # Compute the time difference between the first 'TD5' in the sequence and the test entry's 'TD5'
    time_diff = (
        pd.to_datetime(vessel_sequence["TD5"][0]) - pd.to_datetime(row.time)
    ).total_seconds()

    print(time_diff)

    vessel_sequence["TD5"][-1] = time_diff
    # Append the modified sequence to the list
    modified_sequences.append(vessel_sequence)

# Combine all modified sequences into one DataFrame
sequence_test_data = pd.concat(modified_sequences, ignore_index=True)
array = np.array(sequence_test_data)
print(array.shape)

Index(['TD1', 'TD2', 'TD3', 'TD4', 'TD5'], dtype='object')
Series([], Name: TD5, dtype: object)


KeyError: 0

In [ ]:
np.save("intermediate/test_sequence_data.npy", test_sequence_list)